# Analysing the delays in a morning peak

This notebook takes a all the raw data from a single delay, and determines the number of delays, and the worst delay for a set time period in the day.

In [1]:
import zipfile
import io

In [2]:
file_name = '20190123.zip'
with open(file_name, "rb") as f:
    z = zipfile.ZipFile(io.BytesIO(f.read()))

z.extractall()
print("Extracted file " + file_name)

Extracted file 20190123.zip


We have a few directories to dive into before we get to the good stuff...

In [3]:
import glob
data_path = 'home/pi/sydney-transport-tracker/data/raw/20190123/'
delay_files_count = len(glob.glob(data_path + '*.pickle'))
print('We have ' + str(delay_files_count) + ' delay files')
timetable_files = glob.glob(data_path + '*.txt')
print('The timetable files are: ' + ', '.join(timetable_files))

We have 720 delay files
The timetable files are: home/pi/sydney-transport-tracker/data/raw/20190123/stop_times.txt, home/pi/sydney-transport-tracker/data/raw/20190123/shapes.txt, home/pi/sydney-transport-tracker/data/raw/20190123/stops.txt, home/pi/sydney-transport-tracker/data/raw/20190123/calendar.txt, home/pi/sydney-transport-tracker/data/raw/20190123/trips.txt, home/pi/sydney-transport-tracker/data/raw/20190123/agency.txt, home/pi/sydney-transport-tracker/data/raw/20190123/routes.txt


## What services are running today?
The services are determined in `calendar.txt`, with those service IDs used to filter out everything in `trips.txt`. From there we have trip IDs, which can be used to filter out the scheduled stop times in `stop_times.txt`.

`calendar.txt` -> `service_id` -> `trips.txt` -> `trip_id` -> `stop_times.txt` -> `arrival_time`, `departure_time`

In [4]:
import datetime
import pandas as pd
import csv

day_of_analysis = 'wednesday'
date_of_analysis = datetime.datetime.strptime('20190123', "%Y%m%d").date()
todays_services = []

with open(data_path + 'calendar.txt', mode='r', encoding='utf-8-sig') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        if row[day_of_analysis] == '1':
            start_date = datetime.datetime.strptime(row['start_date'], "%Y%m%d").date()
            end_date = datetime.datetime.strptime(row['end_date'], "%Y%m%d").date()
            if start_date <= date_of_analysis <= end_date:
                todays_services.append(row['service_id'])

print("Todays services = " + ', '.join(todays_services))

Todays services = 1260.122.112, 1260.122.116, 1260.122.120, 1260.122.124, 1260.122.16, 1260.122.20, 1260.122.24, 1260.122.28, 1260.122.48, 1260.122.52, 1260.122.56, 1260.122.60, 1260.122.80, 1260.122.84, 1260.122.88, 1260.122.92


In [5]:
df_trips = pd.read_csv(data_path + 'trips.txt',
                       header=0,
                       encoding='utf-8-sig',
                       usecols=["route_id", "service_id", "trip_id", "trip_short_name"])
df_filtered_trips = df_trips[df_trips['service_id'].isin(todays_services)]
pd.options.display.max_rows = 10
print(df_trips)

       route_id    service_id                         trip_id  trip_short_name
0        BNK_2a    1260.122.4    1--A.1260.122.4.M.8.55188158              NaN
1        BNK_2a   1260.122.48   1--A.1260.122.48.M.8.55188157              NaN
2        BNK_2a   1260.122.64   1--A.1260.122.64.M.8.55188157              NaN
3        BNK_2a    1260.122.8    1--A.1260.122.8.M.8.55188158              NaN
4      RTTA_DEF  1603.103.128  1--A.1603.103.128.M.8.54724494              NaN
...         ...           ...                             ...              ...
57308   CTY_W2a    1620.100.2    WT28.1620.100.2.X.5.55042064              NaN
57309   CTY_W2a    1620.100.4    WT28.1620.100.4.X.5.55042062              NaN
57310   CTY_W2a   483.101.120   WT28.483.101.120.X.5.55277012              NaN
57311   CTY_W2a    487.111.60    WT28.487.111.60.X.5.55310994              NaN
57312   CTY_W2a    487.111.64    WT28.487.111.64.X.5.55310994              NaN

[57313 rows x 4 columns]


Gosh, that's a lot of services... What are RTTA_DEF and RTTA_REV... and are those CountryLink services??? Let's filter out some of these services as we're only going to analyse what is going on in the general Sydney commuter network.

In [6]:
ROUTES_TO_IGNORE = ["CTY_NC1", "CTY_NC1a", "CTY_NC2", 
                    "CTY_NW1a", "CTY_NW1b", "CTY_NW1c", "CTY_NW1d", "CTY_NW2a", "CTY_NW2b", 
                    "CTY_S1a", "CTY_S1b", "CTY_S1c", "CTY_S1d", "CTY_S1e", "CTY_S1f", 
                    "CTY_S1g", "CTY_S1h", "CTY_S1i", 
                    "CTY_S2a", "CTY_S2b", "CTY_S2c", "CTY_S2d", "CTY_S2e", "CTY_S2f", 
                    "CTY_S2g", "CTY_S2h", "CTY_S2i", 
                    "CTY_W1a", "CTY_W1b", "CTY_W2a", "CTY_W2b", 
                    "HUN_1a", "HUN_1b", "HUN_2a", "HUN_2b", 
                    "RTTA_DEF", "RTTA_REV"]
df_trips = df_trips[~df_trips['route_id'].isin(ROUTES_TO_IGNORE)]
print(df_trips)

      route_id   service_id                        trip_id  trip_short_name
0       BNK_2a   1260.122.4   1--A.1260.122.4.M.8.55188158              NaN
1       BNK_2a  1260.122.48  1--A.1260.122.48.M.8.55188157              NaN
2       BNK_2a  1260.122.64  1--A.1260.122.64.M.8.55188157              NaN
3       BNK_2a   1260.122.8   1--A.1260.122.8.M.8.55188158              NaN
6       BNK_2a  1603.103.60  1--A.1603.103.60.M.8.54724492              NaN
...        ...          ...                            ...              ...
57276    BMT_2   483.101.56   WN18.483.101.56.N.2.55278207              NaN
57277    BMT_2   483.101.64   WN18.483.101.64.N.2.55278207              NaN
57278    BMT_2    487.111.4    WN18.487.111.4.N.2.55308164              NaN
57279    BMT_2   487.111.56   WN18.487.111.56.N.2.55308164              NaN
57280    BMT_2   487.111.64   WN18.487.111.64.N.2.55308164              NaN

[47941 rows x 4 columns]


Now, the `stop_times.txt` file contains the stop files from across a number of days. We can filter out which services we want by only looking at trips that are running today.

In [7]:
df_stop_times = pd.read_csv(data_path + 'stop_times.txt', header=0,
                            encoding='utf-8-sig',
                            dtype={'stop_id': str},
                            usecols=["trip_id", "arrival_time", "departure_time", "stop_id"],
                            parse_dates=['arrival_time', 'departure_time'])

# remove any trips from stop_times that did NOT happen on this date
df_filtered_stop_times = df_stop_times[df_stop_times['trip_id'].isin(df_filtered_trips['trip_id'])]
print(df_filtered_stop_times)

                               trip_id arrival_time departure_time  stop_id
10       1--A.1260.122.48.M.8.55188157     03:52:00       03:52:00  2144243
11       1--A.1260.122.48.M.8.55188157     03:54:12       03:55:00  2141313
12       1--A.1260.122.48.M.8.55188157     03:57:30       03:57:30   214063
13       1--A.1260.122.48.M.8.55188157     03:58:42       03:58:42   214074
14       1--A.1260.122.48.M.8.55188157     04:01:24       04:01:24  2135234
...                                ...          ...            ...      ...
1033592  WT28.1260.122.60.X.5.55187037     20:26:24       20:26:24   214072
1033593  WT28.1260.122.60.X.5.55187037     20:27:36       20:29:12  2135232
1033594  WT28.1260.122.60.X.5.55187037     20:31:24       20:31:24   213491
1033595  WT28.1260.122.60.X.5.55187037     20:39:06       20:39:06  2015133
1033596  WT28.1260.122.60.X.5.55187037     20:42:24       23:34:00  2000325

[73135 rows x 4 columns]


## Parsing delay data
In the archive, all of the responses to requests to the [Transport for NSW Open Data API](https://opendata.transport.nsw.gov.au) have been saved. As part of the repository, there is a Python task for making these requests every two minutes, 24 hours a day.
This data is in the format according to [General Transit Feed Specification](https://developers.google.com/transit/) and can be parsed with the [GTFS python library](https://developers.google.com/transit/gtfs-realtime/examples/python-sample).

In [8]:
from google.transit import gtfs_realtime_pb2
import sys
sys.path.append('../')
from src.features.trip_objects import *
import pickle

# get all the delay files
files_in_dir = sorted(glob.glob(data_path + '*.pickle'))

merged_delays = dict()

for delay_data_file in files_in_dir:
    try:
        current_delay_response = pickle.load(open(delay_data_file, "rb"))
    except Exception as e: 
        print(e)
        continue  # next file

    feed = gtfs_realtime_pb2.FeedMessage()
    feed.ParseFromString(current_delay_response)
    for entity in feed.entity:
        if entity.HasField('trip_update') and len(entity.trip_update.stop_time_update) > 0:
            trip_update = TripUpdate(entity.trip_update.trip.trip_id,
                                     entity.trip_update.trip.route_id,
                                     entity.trip_update.trip.schedule_relationship,
                                     entity.trip_update.timestamp)

            for stop_time_update in entity.trip_update.stop_time_update:
                trip_update.stop_time_updates[stop_time_update.stop_id] \
                    = StopTimeUpdate(stop_time_update.stop_id,
                                     stop_time_update.arrival.delay,
                                     stop_time_update.departure.delay,
                                     stop_time_update.schedule_relationship)

            if trip_update is None:
                print('trip update is none')
            # merge with current trips
            if trip_update.trip_id in merged_delays:
                merged_delays[trip_update.trip_id] = \
                    merge_trips(merged_delays[trip_update.trip_id], trip_update)
            else:
                merged_delays[trip_update.trip_id] = trip_update

print("Found " + str(len(merged_delays)) + " trips")

Found 3515 trips
